# Week 4: Machine learning 1

Machine learning is a technique for learning patterns in data that enable computers to make decisions and predictions. It's probably one of the hottest skills to master as a scientist or engineer in research or industry today. This week, we'll get an overview of what machine learning is, what it can be used for and what its limits are. Without worrying too much about what goes on behind the scenes, we will play around with a few classifiers in Python and test model performance using cross validation. The exercises today cover:

* Feature representation
* Model fitting
* Model evaluation
* Prediction results

[**Feedback**](http://ulfaslak.com/vent)

## Exercises

We want to predict whether a character is a hero or a villain from information that we can extract from their markup. This is a large problem that includes some data wrangling, model fitting and a bit of evaluation. Therefore the problem is split into parts.

### Part 1: Feature representation
In it's raw format, the data cannot be given to a machine learning algorithm. What we must do is extract features from the data and put them into a structured format. This is the same as what we did when we looked at a dog (the data) and extracted into a matrix whether it was fluffy, sad looking, etc. (the features). The feature we will extract here is **team alliances**.

We can represent the team alliances of each character as a row in a matrix where each column corresponds to a particular team. That should look something like this (numbers are made up):

<img src="http://ulfaslak.com/computational_analysis_of_big_data/exer_figures/example_boa.png" width="400"/>

**Note**: The following exercises relies on the dataset you produced in Ex. 3.1.2 (character markup stored on your computer). If you didn't manage to produce the dataset [use mine](http://github.com/ulfaslak/caobd_s19/tree/master/data) and go back and complete it at a later time so that you get the most out of this session.

> **Ex.4.1.1**: Write a function called `get_alliances` that takes the name of a character and returns a list of teams that the character is allied with. Print the alliances of Iron Man.

*Hint: There is a place in the character markup where you'll find the team affiliations and you want to write some code that can extract that. The affiliations are not always listed the same way so you should probably try to eyeball some of the documents to get a feel of how the formatting can look.*

In [1]:
import re, os

def get_alliances(char, faction=None):
    """Return list of alliances for Marvel character."""
    
    if faction is None:
        for faction in ["heroes", "ambiguous", "villains"]:
            faction_chars = [c[:-4] for c in os.listdir("./data/%s" % faction)]
            if char in faction_chars:
                break
    
    # Load character markup
    with open("./data/%s/%s.txt" % (faction, char)) as fp:
        markup = fp.read()

    # Get alliance field
    alliances_field = re.findall(r"alliances[\w\W]+?\|.+=", markup)
    if alliances_field == []:
        return []

    # Extract teams from alliance field
    return [t[2:-1] for t in re.findall(r"\[\[.+?[\]\|]", alliances_field[0][10:])]

In [2]:
get_alliances("Iron Man")

['Avengers (comics)',
 'United States Department of Defense',
 'Force Works',
 'The New Avengers (comics)',
 'Guardians of the Galaxy (2008 team)',
 'Illuminati (comics)',
 'The Mighty Avengers',
 'S.H.I.E.L.D.',
 'Stark Industries',
 'Stark Resilient',
 'Thunderbolts (comics)']

> **Ex.4.1.2**: Produce a sorted list called `all_teams` that contains all teams in the entire Marvel universe. Print the first 10.

In [3]:
all_teams=[]
seen = set(all_teams)

for faction in ["heroes", "ambiguous", "villains"]:
    faction_chars = [c[:-4] for c in os.listdir("./data/%s" % faction)]
    
    for char in faction_chars:
        alliances = get_alliances(char)
        
        for team in alliances:
            seen.add(team)

for team in seen:
    all_teams.append(team)

all_teams.sort()    
  
print(len(all_teams))    
print(all_teams[:10])


622
['A-Force', 'A-Next', 'A.R.M.O.R.', 'AXIS (comics)', 'Acolytes (comics)', 'Acts of Vengeance', 'Advanced Idea Mechanics', 'Advanced Idea Mechanics#Avengers Idea Mechanics', 'Age of Apocalypse', 'Agency X']


> **Ex.4.1.3**: Write a function that takes the name of a character and returns a vector representation of the team alliances for that character. The length of the returned list will be `len(all_teams)` and have 0s in the places corresponding to teams that the character is not on, and 1s in the places corresponding to teams the character is on. Print the sum of the list returned when the input to the function is Iron man.

In [4]:
def encode_alliance(char):
    """returns vector of 0,1 whether char is in team"""
    vector = []
    alliances = get_alliances(char)
    c = 0
    
    for team in all_teams:
        if team not in alliances:
            vector.append(0)
        else:
            vector.append(1)
            c+=1
    
    
    return vector, c

In [5]:
im, im_vector = encode_alliance("Iron Man")
print(len(im))
print(im_vector)

622
11


> **Ex. 4.1.4**: Create the team alliance matrix for the data. This is your feature matrix for the classification problem you will solve later in this exercise set. Therefore, you should also—in a seperate *target* array—store whether characters are heros (denote by 1) or villains (denote by 0). For now, skip ambiguous characters, but write your code in such a way that it won't be too hard to redo this for ambiguous characters. Skip characters that have no team alliances. Print the shapes of your matrix and target array.

In [6]:
#create matrix
matrix = []
target = []
for faction in ["heroes", "villains"]:
    faction_chars = [c[:-4] for c in os.listdir("./data/%s" % faction)]
    
    for char in faction_chars:
        vector, length = encode_alliance(char)
        matrix.append(vector)
        
        if faction_chars == "heroes":
            target.append(1)
        else:
            target.append(0)

In [7]:
import numpy as np
#index character on # of teams
print(np.array(matrix).sum(1))

#how many characters per team
print(np.array(matrix).sum(0))

[2 4 1 ... 2 1 0]
[ 6  6  4  1 15  1 18  1  1  1  1  5  1  4  1  1  1  1 11  2  1 19  3  0
  6  2  4  9  5  1  4  1  1  1  4  3  1  1  1 92  1  3 25  1 22  2  2  1
  3  1  1  1  1 10  1  9  1  6  1  1  1  2  1  1  4 37  1  1  7  1  6  1
  1  4  7  8  2  1  1  1  0  1  1 11  9  3  1  1  6  3  1  3  1  7  2  2
  3  1  1  1  1  0  6  4 14  0  6  6  2  2  1  1  1  4  1  2  5  1 57  1
  1  4  2  3  4  1  1  5  1  1  1  1  1  1  1  1  7 10  4  6  1  8 26  1
  0  2  7  1  1  4  1  0  6  7  1  4 22  2  6  1  3  4  2  0  1 14  1 12
  0  2  3  1  6  6  1  1  1  8  1  1  1  1  2 24 14  1  2  1  1  2  5  5
  1 14  1  3  1  1  8  1  2  5 11  1  1 11 20  1  6  2  1  0 11  2  3  3
  3  1  3  1 38  0 19  7 31  1  1  4  5  1 16  3  3  1  1  1  4 29 16  2
  0  2  1  1  8  1  1  8  1  1  1 11  1  1 16  1  1  9  1  1  3  1  2  5
  1  6  5  1  2  2  2  6  1 16  1 34  3  1 14  6  1  1  1  2  3  4  2  1
 21  0  1 22  1  3  4  3  1  3 13  1  1  1  0  2 33  1  1  7 12  1  3  1
  2 61  1  4  8  2  7  2  1  1  1

### Part 2: Model fitting

> **Ex. 4.2.1**: Train a classifier on all of your data and test its accuracy.

>* If your team alliance matrix is `X_ta` and your target array is `y_ta` you can do this by instantiating a model like:
>
        from sklearn.naive_bayes import BernoulliNB
        model = BernoulliNB()
        model.fit(X_ta, y_ta)  # <--- This is the training/fitting/learning step
        
> The `BernoulliNB` is a version of the Naive Bayes classifier which associates certain features with labels and asks what the probability of a label for a data point is given its features. You are free to use any other classifier if you want. Popular ones are trees, random forests, support vector machines, feed forward neural networks, logistic regression, and the list goes on. With `sklearn`, they are just as easy to employ as the `BernoulliNB` classifier.


>1. Test the accuracy of your model. You can use the `.predict` method on the `model` object to get predictions for a matrix of data points. Report the accuracy of your model on the same data that you trained the model on, alongside the baseline accuracy of a "dumb" model that only guesses for the majority class.

>2. Report the precision, recall and F1 scores, with respect to the minority class (heroes). `sklearn` has implementations that you can use if you are short for time. Extra credit for doing it using only basic linear algebra operations with `numpy`, though.

In [ ]:
from sklearn.naive_bayes import BernoulliNB
X_ta = matrix
y_ta = target

model = BernoulliNB()
model.fit(X_ta, y_ta)

In [ ]:
x_test = X_ta[0]
y_test = y_ta[0]

#target
print(y_test)

#prediction
model.predict([x_test])



In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(model, X_ta, y_ta, cv=3)

f1_score(y_ta,y_train_pred)

### Part 3: Model evaluation

> **Ex. 4.3.1**: Investigate how well your model generalizes. You may have noticed that the performance seemed a little too good to be true in Ex 4.2.1.
1. Why did you get such a high accuracy in the previous exercise?
2. Split your data into a test and training set of equal size. Train the model only on the training set and report its accuracy and F1 scores (for both classes) on both the training and test sets.
3. Comment on the difference you observe.

> *Hint: Watch out for unbalanced class proportions! You may want to randomly reorder the rows of your datapoints and target labels so your training and test sets have the same amount of heroes and villains.*

> **Ex. 4.3.2**: Implement cross validation. The performance of a classifier is strongly dependent on the amount of data it is trained on. In Ex. 4.3.1 you train it on only half of the data and test it on the other half. If you rerun that code multiple times, with random 50/50 partitions, you are gonna see a lot of uncertainty in performance. Cross validation solves this problem by training on a larger subset of the data and testing on a smaller one, and taking the average performance over K-folds of this process.
1. Implement cross validation over $K=10$ folds. For each fold you must record the training and test accuracies. In the end, visualize the distributions of test- and training accuracy as histograms in the same plot. It's important that you comment on the result.

### Part 4: Predicting good vs. evil

>**Ex. 4.4.1**: Let's put our classifier to use!
* Retrain your model on all of your data.
* Create a team alliance representation of the ambiguous characters
* Use the model the estimate the probability that each character is a villain (let's call this *villainness*). You can use the `.predict_proba` method on the model to get probability estimates rather than class assignments.
* **Visualize the "heroness" distribution for all ambiguous characters**. Comment on the result.